In [163]:
run mysetup.py

Setup complete.


In [164]:
timestamp   = datetime.datetime.now(ZoneInfo("America/New_York")).strftime("%Y%m%d_%H%M%S")
base_dir    = pathlib.Path.cwd() / "input" / timestamp
base_dir.mkdir(parents=True, exist_ok=True)


In [170]:
# ------------------------------------------------------------------
# 1 ▸ generate a toy instance (10 PICs, 4 houses, 3 sets, 5 groups) --
# ------------------------------------------------------------------

N_PIC = 1000 # number of PICs
N_HOUSE = 90   # number of houses
N_SETS = 4   # number of sets

random.seed(42);  np.random.seed(42)


PIC   = [f'P{i+1}' for i in range(N_PIC)]
HOUSE = [f'H{h+1}' for h in range(N_HOUSE)]
SETS  = [f'SET{s+1}' for s in range(N_SETS)]
MO    = ['MO', 'NO_MO'] # criterion 1: mental observation MO/NO_MO
VIOLENCE    = ['MIN', 'MED', 'MAX'] # criterion 2: violence levels MIN/MED/MAX


# (a) PIC profile  --------------------------------------------------
violence = np.random.choice(VIOLENCE, size=N_PIC)
is_mo    = np.random.choice(MO, size=N_PIC)
violence_df = pd.get_dummies(pd.Series(violence)).reindex(columns=VIOLENCE, fill_value=False).astype(int)
mo_df = pd.get_dummies(pd.Series(is_mo)).reindex(columns=MO, fill_value=False).astype(int)
pic_df = pd.DataFrame({'PIC_ID': PIC})

df_pic = pd.concat([pic_df, mo_df, violence_df], axis=1)

# (b) House profile  ------------------------------------------------
df_init = pd.DataFrame({
    'PIC_ID'        : PIC,
    'HOUSE_ID_INIT' : np.random.choice(HOUSE, size=len(PIC))
})


# (c) Set profile  --------------------------------------------------
membership = np.zeros((len(PIC), len(SETS)), dtype=int)
for i in range(len(PIC)):
    if random.random() < .05: # 20% chance to be in a set
        membership[i, random.randrange(len(SETS))] = 1
df_S = pd.DataFrame(membership, columns=SETS).assign(PIC_ID=PIC)


# (d) random separation-order & SRG conflicts -----------------------
df_so = pd.DataFrame(random.sample(
            list(itertools.combinations(PIC, 2)), 30),
            columns=['PIC_ID1','PIC_ID2'])

df_sc = pd.DataFrame(random.sample(
            list(itertools.combinations(SETS, 2)), 3),
            columns=['SET1','SET2'])

# (e) physical housing & type menus ---------------------------------
df_ph = pd.DataFrame({
    'HOUSE_ID'   : HOUSE,
    'TYPE_BUILD' : np.random.choice(['DORM','CELL'], size=len(HOUSE)),
    'CAP'        : np.random.randint(20,50, size=len(HOUSE))
})





# (f) housing type ---------------------------------


w = {                     # group-permission matrix w_{tℓ}
    'MO_MIN'   : dict(MO=1, NO_MO=0, MIN=1, MED=0, MAX=0),
    'MO_MED'   : dict(MO=1, NO_MO=0, MIN=0, MED=1, MAX=0),
    'MO_MIN_MED'   : dict(MO=1, NO_MO=0, MIN=1, MED=1, MAX=0),
    'NO_MO_MIN': dict(MO=0, NO_MO=1, MIN=1, MED=0, MAX=0),
    'NO_MO_MED': dict(MO=0, NO_MO=1, MIN=0, MED=1, MAX=0),
    'NO_MO_MAX': dict(MO=0, NO_MO=1, MIN=0, MED=0, MAX=1),
    'MO_MAX': dict(MO=1, NO_MO=0, MIN=0, MED=0, MAX=1),
}

TYPES = list(w.keys())
df_tb = df_ph[['HOUSE_ID', 'TYPE_BUILD']].merge(
            pd.DataFrame({'TYPE_ID': TYPES}),
            how='cross'
)

# 2) keep only legal (HOUSE_ID, TYPE_ID) pairs
mask = (
    (df_tb['TYPE_BUILD'] == 'CELL')                           # every TYPE allowed in a CELL
    |                                                         #     ─────────────
    (
        (df_tb['TYPE_BUILD'] == 'DORM')                       # only a subset allowed in a DORM
        & (["MAX" not in item for item in df_tb['TYPE_ID']])
    )
)
df_tb = df_tb.loc[mask, ['HOUSE_ID', 'TYPE_ID']].reset_index(drop=True)



df_tbl = (pd.DataFrame(w).T
          .reset_index(names='TYPE_ID'))


excel_dir = base_dir / "input.xlsx"
with pd.ExcelWriter(excel_dir, engine='xlsxwriter') as writer:
    df_pic .to_excel(writer, sheet_name='df_pic',  index=False)
    df_init.to_excel(writer, sheet_name='df_init', index=False)
    df_S   .to_excel(writer, sheet_name='df_S',    index=False)
    df_so  .to_excel(writer, sheet_name='df_so',   index=False)
    df_sc  .to_excel(writer, sheet_name='df_sc',   index=False)
    df_ph  .to_excel(writer, sheet_name='df_ph',   index=False)
    df_tb  .to_excel(writer, sheet_name='df_tb',   index=False)
    df_tbl .to_excel(writer, sheet_name='df_tbl',  index=False)


/var/folders/v4/ygpx35493v50lqrtm282cqnm0000gn/T/ipykernel_31942/3333987526.py:87: FutureWarning: Logical ops (and, or, xor) between Pandas objects and dtype-less sequences (e.g. list, tuple) are deprecated and will raise in a future version. Wrap the object in a Series, Index, or np.array before operating instead.
  (df_tb['TYPE_BUILD'] == 'DORM')                       # only a subset allowed in a DORM
